# Panel Demo

This notebook demonstrates the use of Holoviz [Panel](https://panel.holoviz.org/index.html). 

*Panel is an open-source Python library that lets you create custom interactive web apps and dashboards by connecting user-defined widgets to plots, images, tables, or text.*

In [ ]:
%matplotlib inline

In [ ]:
import urllib
import os
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from pyproj import CRS
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

import sys
sys.path.append("..")
import movingpandas as mpd

import warnings
warnings.simplefilter("ignore")

import panel as pn
import panel.widgets as pnw

In [ ]:
df = read_file('data/demodata_geolife.gpkg')
df['t'] = pd.to_datetime(df['t'])
df = df.set_index('t').tz_localize(None)

In [ ]:
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id')

In [ ]:
my_traj = traj_collection.trajectories[1].to_crs(CRS(4088))

In [ ]:
my_traj.is_latlon

In [ ]:
def my_plot(tolerance=0.001, variable='douglas-peucker'):
    generalized = mpd.TrajectoryGeneralizer.generalize(my_traj, mode=variable, tolerance=tolerance)
    generalized.add_speed(overwrite=True)
    return ( generalized.hvplot(title='Trajectory {}'.format(generalized.id), c='speed', cmap='Viridis', colorbar=True, clim=(0,20), line_width=10, width=500, height=500) + 
             generalized.df['speed'].hvplot.hist(title='Speed histogram', width=300, height=500) 
           )

my_plot()

In [ ]:
kw = dict(tolerance=(0, 1000), variable=['douglas-peucker', 'min-distance'])
pn.interact(my_plot, **kw)